## Valorant Average Combat Score Prediction Using Dense Neural Network Based on Match Metrics

**Proponents:** *Alforque, Batalan, Verdan*

### Phase 1: Pre-processing

#### Import modules

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.model_selection import train_test_split
from datetime import datetime


#### Define path

In [ ]:
DATASET_PATH = "data/overview.csv"
df = pd.read_csv(DATASET_PATH)
df = df.dropna(subset=['rating', 'acs'])
df.isna().any()
df.shape[0]

#### Extract labels

In [ ]:
y = df['rating'].values
y

#### Select features

In [ ]:
features = ['acs','kills','deaths','assists','kast_percent','adpr','headshot_percent','firstkill','firstdeath','side']
X_raw = df[features].copy()
X_raw.head(5)

#### Check for missing data

In [ ]:
X_raw.isna().any()

#### Impute missing data

In [ ]:
X_raw['kast_percent'].interpolate(method='linear', axis=0, inplace=True)
X_raw['adpr'].interpolate(method='linear', axis=0, inplace=True)
X_raw['headshot_percent'].interpolate(method='linear', axis=0, inplace=True)

In [ ]:
X_raw.isna().any()

#### Split data

In [ ]:
train_X_raw, test_X_raw, train_y, test_y = train_test_split(X_raw, y, test_size=0.2, random_state=0)
print(f'train_X_raw: {train_X_raw.shape}')
print(f'test_X_raw: {test_X_raw.shape}')
print(f'train_y: {train_y.shape}')
print(f'test_y: {test_y.shape}')

In [ ]:
train_X_raw

#### Define encoders

In [ ]:
encoders = ColumnTransformer([
    ('scaler', RobustScaler(), ['acs','kills','deaths','assists','kast_percent','adpr','headshot_percent','firstkill','firstdeath']),
    ('ohe',OneHotEncoder(), ['side'])
])

#### Encode training data

In [ ]:
train_X = encoders.fit_transform(train_X_raw)

In [ ]:
train_X

In [ ]:
train_X.shape

#### Encode test data

In [ ]:
test_X = encoders.transform(test_X_raw)

In [ ]:
test_X.shape

### Phase II: Neural Network Architecture and Rationale

#### Create Keras model

In [ ]:
input_ = tf.keras.layers.Input(shape=(12,))
dense1 = tf.keras.layers.Dense(128, activation='relu')(input_)
reg1 = dense1
reg1 = tf.keras.layers.BatchNormalization()(dense1)
dense2 = tf.keras.layers.Dense(512, activation='relu')(reg1)
dense3 = tf.keras.layers.Dense(256, activation='relu')(dense2)
dense4 = tf.keras.layers.Dense(128, activation='relu')(dense3)
reg2 = dense4
reg2 = tf.keras.layers.BatchNormalization()(dense4)
dense5 = tf.keras.layers.Dense(64, activation='relu')(reg2)
output = tf.keras.layers.Dense(1, activation='linear')(dense5)

In [ ]:
model = tf.keras.Model(inputs=input_, outputs=output)

In [ ]:
model.summary()

### Phase III: Training Methods

#### Compile model

In [ ]:
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['root_mean_squared_error','mean_absolute_error', 'mean_absolute_percentage_error','r2_score'
    ]
)

#### Define early stop

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)

#### Define model checkpoint

In [ ]:
MODEL_PATH = 'finalprojcheckpoint2/checkpoints_it_9/model_at_{epoch:02d}.keras'
os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(MODEL_PATH)

#### Train model

In [ ]:
training_history = model.fit(train_X, train_y, batch_size=64, epochs=50, validation_split=0.2, callbacks=[early_stop, model_checkpoint])

#### Save model

In [ ]:
SAVED_MODEL_PATH = 'finalprojcheckpoint2/checkpoints_it_9/model.keras'
model.save(SAVED_MODEL_PATH)

### Phase IV: Post Processing

#### Load and verify model

In [ ]:
saved_model = tf.keras.models.load_model(SAVED_MODEL_PATH)
saved_model.summary()

#### Consume best model

In [ ]:
test_y_pred = saved_model.predict(test_X)

In [ ]:
results = saved_model.evaluate(test_X, test_y, verbose=1)

#### Plot model

In [ ]:
img_file = 'finalprojcheckpoint2/checkpoints_it_9/model.png'
tf.keras.utils.plot_model(model, to_file=img_file, dpi=200, show_shapes=True, show_dtype=True, show_layer_names=True,show_layer_activations=True, show_trainable=True)